# Objetivo: Previsão de pacientes na UTI (COVID-19)

Utilizarei a base de dados do Hospital Sírio Libanês. A base de dados esta disponível no site do [Kaggle](https://www.kaggle.com/) na pagina do grupo do Sírio Libanês [COVID-19 - Clinical Data to assess diagnosis](https://www.kaggle.com/S%C3%ADrio-Libanes/covid19).

Essa base de dados contém informações, não sensíveis, que diz respeito a quantidade de pacientes que foram ou não internados por covid-19 na clínica do hospital durante a pandemia de corona virus. As informações são ricas com respeito ao quadro clínico e a pergunta que vamos tentar responder é: 

> **Dado um novo paciente conseguiremos prever a chance dele ser encaminhado para a UTI?**


In [1]:
import pandas as pd

In [2]:
sirio_libanes = pd.read_excel('dados/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx')
sirio_libanes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1925 entries, 0 to 1924
Columns: 231 entries, PATIENT_VISIT_IDENTIFIER to ICU
dtypes: float64(225), int64(4), object(2)
memory usage: 3.4+ MB


In [3]:
sirio_libanes.sample(5).T

,728,877,599,1816,939
PATIENT_VISIT_IDENTIFIER,145,175,119,363,187
AGE_ABOVE65,1,0,0,1,1
AGE_PERCENTIL,Above 90th,30th,20th,90th,70th
GENDER,1,1,0,0,0
DISEASE GROUPING 1,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
RESPIRATORY_RATE_DIFF_REL,-0.634409,NaN,0.206452,NaN,-0.939068
TEMPERATURE_DIFF_REL,-0.906821,NaN,-0.340619,NaN,-0.928473
OXYGEN_SATURATION_DIFF_REL,-0.980232,NaN,0.513836,NaN,-0.940077
WINDOW,6-12,4-6,ABOVE_12,2-4,ABOVE_12


## Tarefas impostas pelo Kaggle 

> **Task 1**: Preve a admissão na UTI os casos confirmados de covid-19. Com base na amostra acima, verificar a viabilidade de previsão/classificação dos pacientes que precisarão ou não do suporte de terapia intensiva do hospital. O objetivo é fornecer ao hospital uma previsão de quatro ou três semenas de antecedência da forma mais acurada possível para que os recursos utilizados na UTI possam ser arranjados ou remanejados. 

> **Task 2**: Prever a **NÃO** admissão dos casos de covid-19 na UTI. Com base na amostra dos dados, prever quais pacientes não preciarão de suporte a unidade de terapia intensiva. O objetivo é fornecer aos hospitais temporários e locais uma resposta boa o suficiente para que os médicos de linha de frente possam dar alta com segurança e acompanhar pacientes remotamente.

Note que o objetivo não se reduz a fazer uma análise de estatistica descritiva, o objetivo é classificação de pacientes que precisarão de uma UTI ou não. Ou seja, **uma classificação binária.**

---

## Vamos verificar qual a quantidade dos dados que foram ou não pra UTI:

In [13]:
sirio_libanes['ICU'].value_counts(normalize=True).round(2)*100

0    73.0
1    27.0
Name: ICU, dtype: float64

### Comentário sobre a proporção de pacientes que precisaram da UTI
> Acima podemos verificar que nesta base de dados, dentre os 1924 registros, a quantidade de pessoas que evoluem até a situação em que necessitam de uma UTI é de quase 27%, os outros 73% precisaram do serviço.

## A respeito da anonimização dos dados

A Lei Geral de Proteção de Dados Pessoais (LGPD), Lei nº 13.709, de 14 de agosto de 2018, dispõe sobre o tratamento de dados pessoais, inclusive nos meios digitais, por pessoa natural ou por pessoa jurídica de direito público ou privado, com o objetivo de proteger os direitos fundamentais de liberdade e de privacidade e o livre desenvolvimento da personalidade da pessoa natural. 

Tendo em vista a vigência dessa Lei, de suma importancia pra sociedade, é importante que os dados sejam anonimizados para respeitar os direitos dos cidadãos brasileiros. É importante citar isso pois anonimização de um dado deve ser muito bem feita afim de não violar o direito dos individuos.

Note que só o fato de sabermos que a mostra foi retirada dentre os pacientes do Hospital Sírio Libanês em um intervalo X de tempo já reduz muito o fator de anonimidade do dado e se uma variável fosse explicitamente a comorbidade "diabetes", "pressão alta" ou "HIV", caso as variáveis não fossem *clusterizadas*, sabendo a prevalência de uma doença numa população é possível encontrar as características de cada amostra e é importante dificultar este tipo de análise para estar de acordo com a lei de proteção de dados.

Uma forma de anonimizar esses dados é criar grupos de características entre as amostras evitando assim de informar exatamente quais são as características de um elemento da base de dados. 

fonte: [Lei Geral de Proteção de Dados - LGPD](https://www.gov.br/defesa/pt-br/acesso-a-informacao/lei-geral-de-protecao-de-dados-pessoais-lgpd)

## Propondo modelo Linear

Um modelo matemático pode ser expresso em uma função, tal que: 

### $$\vec{y} = \vec{f}(\vec{x})$$

no caso $\vec{f}$  é um operador vetorial que irá realizar alguma transformação nos dados $\vec{x}$ resultando em $\vec{y}$. Note que em geral temos $\vec{x}$ e $\vec{y}$ e queremos ajustar um modelo $\vec{f}$.

### Dados de treino e dados de teste

Técnicas de regressão em geral tentam ajustar uma curva a um conjunto de dados e posteriormente após ter a curva parte da análise é entender como variam os **resíduos** que no caso é a diferença entre um dado e o resultado predito pelo modelo. Em aprendizado de máquina esta técnica também é utilizada porém com uma modificação.

Os dados que serão fornecidos pelo modelo são classificados em **dados de treino** e **dados de teste**. Aos dados de treino a técnica é exatamente a mesma usada em técnica de regressões: 

> Usa $x$ **entradas de treino** pra ajustar um modelo $f$ que vai calcular uma **saida** $y'$ prevista pelo modelo. Então compara-se $y$ real com $y'$ previsto.

A introdução do conceito dos **dados de teste** é garantir que mesmo que com $x$ dados de entrada nunca vistos o modelo $f$ consegue ainda assim abstrair novas informações nunca vistas afim de estimar $y'$ cujo o resultado é o melhor valor de $y$.

> Usa $x$ entradas de treino pra ajustar um modelo $f$ que vai calcular uma saida $y'$ prevista pelo modelo. Então compara-se $y$ real com $y'$ previsto. Agora afim de testar a abstração do modelo, fornecemos $X$ **entradas de teste** para verificar a precisão de $f$ ao gerar $Y'$ **saidas previstas** em relação aos $Y$ **saidas de teste**

Um ponto importante é que a abstração do modelo esta sempre quantificada em relação aos dados de teste, não aos dados de treino. Isso porque os dados de treino são usados para treinar e estimar um modelo, já os dados de teste são dados que o modelo não teve contato e portanto as informações do $X_{teste}$ são novas ao modelo então precisamos verificar o quão bom o modelo é pra para estimar resultados em cima de dados que ele nunca viu antes. Uma forma bem simples de entender esse algorítmo de treinamento e teste é: 

1. Separa a base de dados em dados de treino e dados de teste.
2. Usa os dados de treino ($x,y$) para ajustar $f$.
3. Usa o modelo $f$ ajustado em dados de teste.
4. Comparar sempre o resultado em relação aos dados de teste.

In [36]:
colunas_quantitativas = sirio_libanes.describe().columns

sirio_libanes_quantitativos = sirio_libanes[colunas_quantitativas].dropna()
sirio_libanes_quantitativos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 545 entries, 4 to 1924
Columns: 229 entries, PATIENT_VISIT_IDENTIFIER to ICU
dtypes: float64(225), int64(4)
memory usage: 979.3 KB


In [37]:
y = sirio_libanes_quantitativos['ICU']
x = sirio_libanes_quantitativos.drop('ICU', axis=1)

In [38]:
display(y.shape)
display(x.shape)

(545,)

(545, 228)

### Modelo de Regressão Logística

A regressão logística é uma técnica estatística que tem como objetivo produzir, a partir de um conjunto de observações, um modelo que permita a predição de valores tomados por uma variável categórica, frequentemente binária, a partir de uma série de variáveis explicativas contínuas e/ou binárias.

* Em medicina, permite por exemplo determinar os factores que caracterizam um grupo de indivíduos doentes em relação a indivíduos sãos.
* No domínio dos seguros, permite encontrar fracções da clientela que sejam sensíveis a determinada política securitária em relação a um dado risco particular.
* Em instituições financeiras, pode detectar os grupos de risco para a subscrição de um crédito.
* Em econometria, permite explicar uma variável discreta, como por exemplo as intenções de voto em actos eleitorais.

#### Considerações do modelo: 
* Relação linear entre o vetor das variáveis explicativas X e o logit da variável resposta Y
* Ausência de multicolinearidade
* Valor esperado dos resíduos igual a zero
* Ausência de heterocedasticidade
* Não pressupõe normalidade dos resíduos nem homogeneidade de variâncias. 

#### SK Learn
```python
sklearn.linear_model.LogisticRegression(penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
```
[[source]](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [48]:
from sklearn.linear_model import LogisticRegression

In [49]:
modelo_regressao_logistica = LogisticRegression()

In [50]:
modelo_regressao_logistica.fit(x,y)

AttributeError: 'str' object has no attribute 'decode'